### Napovedovanje okužb
Naloge se lotim z nevronskimi mrežami. Vhodna plast je velikosti N, izhodna velikosti M. 

Obliko mreže bomo izbrali s pomočjo algoritmov za hiperoptimizacijo. 

In [166]:
import pandas as pd
import torch
from torch.nn.functional import normalize
import numpy as np

In [167]:
from prepeare_data import transform_data
data = pd.read_csv('okuzeni.csv')
n_obcin = data.shape[1]
data.head()

MESTNE_OBCINE = [
'ljubljana',
'maribor',
'kranj',
'koper',
'celje',
'novo_mesto',
'velenje',
'nova_gorica',
'krško',
'ptuj',
'murska_sobota',
'slovenj_gradec'
]
N_MESTNIH_OBCIN = len(MESTNE_OBCINE)



Imamo podatke za vseh 192 občin. Mnogo podatkov manjka. Prav tako mislim, da ne potrebujemo upoštevati vseh občin. 

Odločil sem se, da bom občine z več kot 50imi manjkajočimi vrednosti odstranil iz analize, pri ostalih pa bom manjkajoče vrednosti interpoliral glede na sosednje dni

In [168]:
n_nan = data.isna().sum()
#n_nan.plot()
print( "Število odstranjenih občin:" , len(n_nan[n_nan > 50]))
# odstranimo "slabe" obcine
obcine = set(data.columns) - set(n_nan[n_nan > 50].index)
obcine = list(obcine)
n_obcin = len(obcine)

#nismo odstranili slabih:
#>>> for o in MESTNE_OBCINE:
#        print(o in obcine)
#True

# posodobimo podatke

data = data[obcine]
# interpolacija vzdolž stolpcev:
data = data.interpolate(axis=1)
# linearna interpolacija pusti Nane na začetku -- te zafilamo 
data = data.fillna(method='bfill')

print("Število manjkajočih vrednosti: ", data.isna().sum().sum())

Število odstranjenih občin: 43
Število manjkajočih vrednosti:  0


In [169]:
%reload_ext autoreload
%autoreload 2
from prepeare_data import transform_data
# settings:
N=14
M = 7

X, y = transform_data(data, N, M, MESTNE_OBCINE)
X.shape

torch.Size([779, 14, 150])

## Optimizacija parametrov

In [170]:
from models import fully_connected, convolution, double_convolution, nn_test
linear = fully_connected(M=7,layer_sizes=[N*n_obcin, 500], n_obcin=n_obcin)
conv = convolution(M=7, c=5, l=2, p=(2,3), n_obcin=n_obcin)
double_conv = double_convolution(7, 5, 2, 2,2,(2,3), (1,1), n_obcin=n_obcin)